In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import urllib.request
import time 

In [2]:
# Create lists to store the scraped data 
dates = []
statements = []
sources = []
targets = [] 
platform=[]

In [3]:
def scrape_website(page_number):


    page_num = str(page_number)  
    URL = ('https://www.politifact.com/factchecks/list/?page='+page_num+'&category=truth-o-meter')
    webpage = requests.get(URL)  
    time.sleep(3)
    soup = BeautifulSoup(webpage.text, "html.parser") 
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'}) 
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})
    target = soup.find_all('div', attrs={'class':'m-statement__meter'}) 

    for i in statement_quote:
        link2 = i.find_all('a')
        statements.append(link2[0].text.strip())
        
    for i in statement_meta:
        link3 = i.find_all('a') 
        source_text = link3[0].text.strip()
        sources.append(source_text)
        
    for i in statement_meta:
        link4 = i.find_all('div') 
        date_init = link4[0].text.strip()
        date_text=date_init.split(" ")[2]+date_init.split(" ")[3]+date_init.split(" ")[4]
        dates.append(date_text)
        
        
    for i in statement_meta:
        try:
            link5 = i.find_all('div') 
            platform_init = link5[0].text.strip()
            platform_text=platform_init.split(" in ")[1]
            platform.append(platform_text.strip(":"))
        except IndexError:
            platform.append(None)
        
    for i in target:
        fact = i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        if fact=='half-true':
            targets.append(None)
        else:
            targets.append(fact)
        

In [4]:
n=200
for i in range(1, n):
    scrape_website(i)


data = pd.DataFrame(columns = ['date',  'platform', 'text','person', 'truthfulness category']) 
data['date'] = dates
data['platform'] = platform
data['text'] = statements
data['person'] = sources
data['truthfulness category'] = targets
#delete the row with truthfulness category='half-true'
data=data.dropna(axis=0, how='any')


In [5]:
data.to_csv("data.csv", index= False, encoding= 'utf-8')
df=pd.read_csv("data.csv", encoding= 'utf-8')
df.head()

,date,platform,text,person,truthfulness category
0,"November28,2022",a post,The 2021 infrastructure bill requires all new ...,Instagram posts,barely-true
1,"November22,2022",Public statement,“A lot of the programs (that were) started dur...,Robin Vos,barely-true
2,"November17,2022",a post,“Joe Biden is being investigated for human tra...,Instagram posts,false
3,"November19,2022",a speech,Herschel Walker “wants a nationwide ban on abo...,Raphael Warnock,mostly-true
4,"November20,2022",a tweet,The Club Q shooting was a “false flag.”,Tweets,pants-fire
